In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
project_path = '/content/drive/MyDrive/Colab Notebooks/PatternMining'

In [4]:
import pandas as pd
import numpy as np
small_data = pd.read_csv(f'{project_path}/datasets/all_video_games(cleaned).csv')
big_data = pd.read_csv(f'{project_path}/datasets/vgchartz-2024.csv').rename(columns=str.lower).rename(columns={'release_date': 'release date'})


In [5]:
import json

expanded_rows = []

# Iterate through each row in the DataFrame
for idx, row in small_data.iterrows():
    platforms_info = row['Platforms Info']
    platforms_info = platforms_info.replace("'", "\"")
    platforms = json.loads(platforms_info)

    # Iterate through each platform in the list
    for platform in platforms:
        # print(platform)
        platform_name = platform['Platform']
        metascore = platform['Platform Metascore']
        metascore_count = platform['Platform Metascore Count']

        # Append the expanded row to the list

        row = list(row)
        expanded_rows.append(row[:-1] + [platform_name, metascore, metascore_count])

columns_list = list(small_data.columns)
columns_list.remove('Platforms Info')

edited_small_data = pd.DataFrame(expanded_rows, columns=columns_list + ['platform', 'metascore', 'metascore_count'])

small_data = edited_small_data.rename(columns=str.lower)

In [7]:
small_data['metascore_count'] = small_data['metascore_count'].str.extract(r'(\d+)', expand=False).fillna('')


In [8]:
small_data = small_data.drop(columns=['genres splitted'])

In [9]:
print(small_data['platform'].unique())

print(big_data['console'].unique())


['iOS (iPhone/iPad)' 'Xbox' 'PC' 'GameCube' 'PlayStation 2' 'Wii'
 'PlayStation 3' 'Xbox 360' 'PSP' 'PlayStation 4' 'PlayStation Vita'
 'Xbox One' 'Nintendo Switch' 'DS' 'Game Boy Advance' 'PlayStation 5'
 'Meta Quest' '3DS' 'Wii U' 'Xbox Series X' 'PlayStation' 'Nintendo 64'
 'Dreamcast']
['PS3' 'PS4' 'PS2' 'X360' 'XOne' 'PC' 'PSP' 'Wii' 'PS' 'DS' '2600' 'GBA'
 'NES' 'XB' 'PSN' 'GEN' 'PSV' 'DC' 'N64' 'SAT' 'SNES' 'GBC' 'GC' 'NS'
 '3DS' 'GB' 'WiiU' 'WS' 'VC' 'NG' 'WW' 'SCD' 'PCE' 'XBL' '3DO' 'GG' 'OSX'
 'Mob' 'PCFX' 'Series' 'All' 'iOS' '5200' 'And' 'DSiW' 'Lynx' 'Linux' 'MS'
 'ZXS' 'ACPC' 'Amig' '7800' 'DSi' 'AJ' 'WinP' 'iQue' 'GIZ' 'VB' 'Ouya'
 'NGage' 'AST' 'MSD' 'S32X' 'XS' 'PS5' 'Int' 'CV' 'Arc' 'C64' 'FDS' 'MSX'
 'OR' 'C128' 'CDi' 'CD32' 'BRW' 'FMT' 'ApII' 'Aco' 'BBCM' 'TG16']


In [10]:
mapping = {
    'iOS (iPhone/iPad)': 'iOS',
    'Xbox': 'XB',
    'PC': 'PC',
    'GameCube': 'GC',
    'PlayStation 2': 'PS2',
    'Wii': 'Wii',
    'PlayStation 3': 'PS3',
    'Xbox 360': 'X360',
    'PSP': 'PSP',
    'PlayStation 4': 'PS4',
    'PlayStation Vita': 'PSV',
    'Xbox One': 'XOne',
    'Nintendo Switch': 'NS',
    'DS': 'DS',
    'Game Boy Advance': 'GBA',
    'PlayStation 5': 'PS5',
    '3DS': '3DS',
    'Wii U': 'WiiU',
    'Xbox Series X': 'Series',
    'PlayStation': 'PS',
    'Nintendo 64': 'N64',
    'Dreamcast': 'DC'
}

small_data['platform'] = small_data['platform'].replace(mapping)

In [11]:
big_data = big_data.rename(columns={'console': 'platform'})
data = small_data.merge(big_data, on=['title', 'platform'], how='inner')

In [12]:
print(data['genre'].isnull().sum())
print(data['genres'].isnull().sum())


# az anja ke har do sotun khali nadarand pas yeki az anha ra hazf mikonim
# az anja ke sotun genre kameltar ast, behtar ast genres ra drop konim
data = data.drop(columns=['genre'])

0
0


In [13]:
print(data['developer_x'].isnull().sum())
print(data['developer_y'].isnull().sum())

#az anja ke developer_y null kamtari darad
# va ba moshahede data mafahmin jozeeat bishtari darad pas developer_x ra drop mikonim
data = data.drop(columns=['developer_x'])

11
2


In [14]:
print(data['publisher_x'].isnull().sum())
print(data['publisher_y'].isnull().sum())

#az anja ke publisher_y null kamtari darad
# va ba moshahede data mafahmin jozeeat bishtari darad pas publisher_x ra drop mikonim
# be tor kolli sotoon haye x daraye jozeeat kamtari and chon hasel expand shodan yek recordand ke dar y ingune nist

data = data.drop(columns=['publisher_x'])

11
0


In [15]:
data

,title,release date_x,genres,product rating,user score,user ratings count,platform,metascore,metascore_count,img,publisher_y,developer_y,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release date_y,last_update
0,4X4 EVO 2,11/15/2001,Auto Racing Sim,Rated E For Everyone,NaN,NaN,XB,59,15,/games/boxart/5011937ccc.jpg,Gathering of Developers,Terminal Reality,NaN,0.24,0.18,NaN,0.05,0.01,2001-11-14,NaN
1,4X4 EVO 2,11/15/2001,Auto Racing Sim,Rated E For Everyone,NaN,NaN,GC,56,7,/games/boxart/6083195ccc.jpg,VU Games,Terminal Reality,5.1,0.14,0.11,NaN,0.03,0.00,2002-09-09,NaN
2,Gothic 3,11/14/2006,Western RPG,Rated T For Teen,7.5,832.0,PC,63,39,/games/boxart/3146876ccc.jpg,Aspyr,Piranha Bytes,6.8,NaN,NaN,NaN,NaN,NaN,2006-11-14,NaN
3,Guitar Hero III: Legends of Rock,10/28/2007,Rhythm,Rated T For Teen,8.4,144.0,Wii,86,36,/games/boxart/full_2081446AmericaFrontccc.jpg,RedOctane,Vicarious Visions,8.6,4.60,3.04,0.00,1.12,0.44,2007-10-28,NaN
4,Guitar Hero III: Legends of Rock,10/28/2007,Rhythm,Rated T For Teen,8.4,144.0,PS2,82,24,/games/boxart/full_5372169AmericaFrontccc.jpg,RedOctane,BudCat Creations,8.2,4.98,3.49,0.01,0.01,1.48,2007-10-28,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15008,Inversion,6/5/2012,Third Person Shooter,Rated M For Mature,6.4,81.0,PS3,56,23,/games/boxart/full_inversion_773PALFront.jpg,Namco Bandai Games,Saber Interactive,NaN,0.07,0.03,NaN,0.03,0.01,2012-06-05,NaN
15009,Alice: Madness Returns,6/14/2011,Survival,Rated M For Mature,8.3,276.0,PC,75,29,/games/boxart/full_alice-madness-returns_416Am...,Electronic Arts,Spicy Horse,6.7,0.12,NaN,NaN,0.10,0.01,2011-06-14,NaN
15010,Alice: Madness Returns,6/14/2011,Survival,Rated M For Mature,8.3,276.0,PS3,70,51,/games/boxart/full_alice-madness-returns_955Am...,Electronic Arts,Spicy Horse,6.7,0.59,0.22,0.04,0.24,0.08,2011-06-14,NaN
15011,Alice: Madness Returns,6/14/2011,Survival,Rated M For Mature,8.3,276.0,X360,70,67,/games/boxart/full_alice-madness-returns_827Am...,Electronic Arts,Spicy Horse,6.9,0.50,0.27,0.04,0.15,0.04,2011-06-14,NaN


In [16]:
print(data['release date_x'].isnull().sum())
print(data['release date_y'].isnull().sum())

#az anja ke release date_x null kamtari darad
# va ba moshahede data mafahmin release date_y jozeeat bishtari darad pas
# jaee ke release date_y null ast meghdar release date_x re migozarim
# be tor kolli sotoon haye x daraye jozeeat kamtari and chon hasel expand shodan yek recordand ke dar y ingune nist

data['release date_y'].fillna(data['release date_x'], inplace=True)
data = data.drop(columns=['release date_x'])

31
1900


In [17]:
for column in data.columns:
    if '_y' in column:
        data = data.rename(columns={column: column[:-2]})

In [18]:
data

,title,genres,product rating,user score,user ratings count,platform,metascore,metascore_count,img,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release date,last_update
0,4X4 EVO 2,Auto Racing Sim,Rated E For Everyone,NaN,NaN,XB,59,15,/games/boxart/5011937ccc.jpg,Gathering of Developers,Terminal Reality,NaN,0.24,0.18,NaN,0.05,0.01,2001-11-14,NaN
1,4X4 EVO 2,Auto Racing Sim,Rated E For Everyone,NaN,NaN,GC,56,7,/games/boxart/6083195ccc.jpg,VU Games,Terminal Reality,5.1,0.14,0.11,NaN,0.03,0.00,2002-09-09,NaN
2,Gothic 3,Western RPG,Rated T For Teen,7.5,832.0,PC,63,39,/games/boxart/3146876ccc.jpg,Aspyr,Piranha Bytes,6.8,NaN,NaN,NaN,NaN,NaN,2006-11-14,NaN
3,Guitar Hero III: Legends of Rock,Rhythm,Rated T For Teen,8.4,144.0,Wii,86,36,/games/boxart/full_2081446AmericaFrontccc.jpg,RedOctane,Vicarious Visions,8.6,4.60,3.04,0.00,1.12,0.44,2007-10-28,NaN
4,Guitar Hero III: Legends of Rock,Rhythm,Rated T For Teen,8.4,144.0,PS2,82,24,/games/boxart/full_5372169AmericaFrontccc.jpg,RedOctane,BudCat Creations,8.2,4.98,3.49,0.01,0.01,1.48,2007-10-28,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15008,Inversion,Third Person Shooter,Rated M For Mature,6.4,81.0,PS3,56,23,/games/boxart/full_inversion_773PALFront.jpg,Namco Bandai Games,Saber Interactive,NaN,0.07,0.03,NaN,0.03,0.01,2012-06-05,NaN
15009,Alice: Madness Returns,Survival,Rated M For Mature,8.3,276.0,PC,75,29,/games/boxart/full_alice-madness-returns_416Am...,Electronic Arts,Spicy Horse,6.7,0.12,NaN,NaN,0.10,0.01,2011-06-14,NaN
15010,Alice: Madness Returns,Survival,Rated M For Mature,8.3,276.0,PS3,70,51,/games/boxart/full_alice-madness-returns_955Am...,Electronic Arts,Spicy Horse,6.7,0.59,0.22,0.04,0.24,0.08,2011-06-14,NaN
15011,Alice: Madness Returns,Survival,Rated M For Mature,8.3,276.0,X360,70,67,/games/boxart/full_alice-madness-returns_827Am...,Electronic Arts,Spicy Horse,6.9,0.50,0.27,0.04,0.15,0.04,2011-06-14,NaN


In [20]:
data.to_csv(f'{project_path}/datasets/data.csv', index=False)
